In [1]:
# ⚙️ Überprüfen, ob CUDA verfügbar ist (für GPU-Beschleunigung)
import torch

print("CUDA verfügbar:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU nicht verfügbar, das Training wird auf der CPU ausgeführt.")


CUDA verfügbar: True
GPU: Tesla T4


In [4]:
!pip install transformers datasets wandb torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# 📥 Laden des WikiText-2 Datensatzes
# Wir verwenden die "datasets"-Bibliothek von Hugging Face.

from datasets import load_dataset

# Den Datensatz laden (train, validation, test)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Einen Blick auf den Datensatz werfen
print(dataset)

# Beispielausgabe anzeigen
print(dataset['train'][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})
{'text': ''}


In [ ]:
print(dataset['train'][:20])

{'text': ['', ' = Valkyria Chronicles III = \n', '', ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n', " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game m

In [7]:
# 🧩 Initialisierung des Tokenizers
# Wir verwenden den GPT-2 Tokenizer aus Hugging Face.

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT-2 hat keinen PAD-Token, also fügen wir ihn manuell hinzu
tokenizer.pad_token = tokenizer.eos_token




tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Test: Einen Beispielsatz tokenisieren
example = dataset['train'][1]['text']
print(example)
tokens = tokenizer.encode(example)
print("Tokenisierte Beispieldaten:", tokens)

 = Valkyria Chronicles III = 

Tokenisierte Beispieldaten: [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198]


In [8]:
# 🔄 Tokenisierung und Vorbereitung des Datensatzes für das Training
from torch.utils.data import DataLoader

block_size = 128  # Länge der Input-Sequenz

def tokenize_function(angabe):
    # Den Text in Token umwandeln
    return tokenizer(angabe["text"])

# Tokenisierung anwenden
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Beispiele anzeigen
print(tokenized_datasets["train"][0])

# Funktion, um die Token in Blöcke der Länge block_size zu teilen
def group_texts(angabe):
    # Alle Token zusammenfügen
    concatenated = {k: sum(angabe[k], []) for k in angabe.keys()}
    total_length = len(concatenated[list(angabe.keys())[0]])

    # Länge kürzen, damit sie durch block_size teilbar ist
    total_length = (total_length // block_size) * block_size

    # In Blöcke aufteilen
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }

    # Die Labels sind identisch zu den Inputs für autoregressive Modelle
    result["labels"] = result["input_ids"].copy()
    return result

# Die Funktion anwenden
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
def collate_fn(batch):
    # Pad input_ids and labels to the maximum length in the batch
    max_len = max(len(x['input_ids']) for x in batch)
    input_ids = [x['input_ids'] + [tokenizer.pad_token_id] * (max_len - len(x['input_ids'])) for x in batch]
    labels = [x['labels'] + [-100] * (max_len - len(x['labels'])) for x in batch]  # -100 is the ignore index for the loss function

    # Convert to tensors
    input_ids = torch.tensor(input_ids)
    labels = torch.tensor(labels)

    return {'input_ids': input_ids, 'labels': labels}
train_dataloader = DataLoader(lm_datasets["train"], batch_size=8, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(lm_datasets["validation"], batch_size=8, collate_fn=collate_fn)

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

{'input_ids': [], 'attention_mask': []}


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [13]:
#  Modelldefinition
import torch.optim as optim
import torch.nn as nn
import wandb # Import the wandb library

class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=2, num_layers=2):
        super().__init__()
        # Token-Embedding
        self.embedding = nn.Embedding(vocab_size, d_model)

        # Transformer Decoder Layer
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # Lineare Projektion auf die Vokabulargröße
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids):
        # Eingabe-Token zu Embeddings
        embedded = self.embedding(input_ids).transpose(0, 1)  # (Seq_len, Batch, D_model)

        # Causal Masking erstellen
        seq_len = embedded.size(0)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(input_ids.device)

        # Transformer Decoder anwenden
        decoded = self.transformer_decoder(embedded, embedded, tgt_mask=tgt_mask)

        # Projektion auf Vokabular
        logits = self.output_layer(decoded).transpose(0, 1)  # (Batch, Seq_len, Vocab_size)
        return logits

#  Modell initialisieren
# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DecoderOnlyTransformer(vocab_size=len(tokenizer)).to(device)

#  Optimizer und Loss-Funktion
optimizer = optim.AdamW(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

#  Trainings-Funktion
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in dataloader:
        # Daten auf das Gerät verschieben
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        # Gradienten zurücksetzen
        optimizer.zero_grad()

        # Vorwärtsdurchlauf
        outputs = model(input_ids)

        # Berechnung des Losses (wir ignorieren Padding-Tokens)
        loss = criterion(outputs.reshape(-1, outputs.size(-1)), labels.reshape(-1)) # Use reshape instead of view

        # Rückwärtsdurchlauf
        loss.backward()

        # Parameter aktualisieren
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# 🧪 Evaluierungs-Funktion
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), labels.reshape(-1))
            total_loss += loss.item()


    return total_loss / len(dataloader)


# Initialisierung von wandb für das Loggen der Trainings- und Validierungsdaten
wandb.init(project="GenAI_project", entity="flora-nuta-hochschule-hannover")
epochs = 5
for epoch in range(epochs):
    train_loss = train(model, train_dataloader, optimizer, criterion)
    eval_loss = evaluate(model, eval_dataloader, criterion)
    print(f"Epoche {epoch + 1}: Train Loss = {train_loss:.4f}, Eval Loss = {eval_loss:.4f}")
    # Logge die Verluste (Training und Validierung) zu wandb
    wandb.log({"train_loss": train_loss, "eval_loss": eval_loss})

Epoche 1: Train Loss = 1.3237, Eval Loss = 0.5293
Epoche 2: Train Loss = 0.0910, Eval Loss = 0.1935
Epoche 3: Train Loss = 0.0215, Eval Loss = 0.1346
Epoche 4: Train Loss = 0.0067, Eval Loss = 0.1190
Epoche 5: Train Loss = 0.0010, Eval Loss = 0.1237


In [17]:
import os

!find / -name "genai_model.bin"

find: ‘/proc/66/task/66/net’: Invalid argument
find: ‘/proc/66/net’: Invalid argument


# Neuer Abschnitt

In [14]:
from huggingface_hub import HfApi, HfFolder, Repository, create_repo
import os

# Lokales Verzeichnis für das Modell
model_dir = "decoder-only-transformer"

# Verzeichnis erstellen
os.makedirs(model_dir, exist_ok=True)

# Modell und Tokenizer speichern
torch.save(model.state_dict(), os.path.join(model_dir, "genai_model.bin"))
tokenizer.save_pretrained(model_dir)

# Optional: Konfigurationsdatei speichern
config = {
    "architectures": ["DecoderOnlyTransformer"],
    "vocab_size": len(tokenizer),
    "block_size": block_size,
    "model_type": "decoder-only",
}
import json
with open(os.path.join(model_dir, "config.json"), "w") as f:
    json.dump(config, f)


In [15]:
from huggingface_hub import upload_folder

# Name des Repos, z.B. dein_username/decoder-only-transformer
repo_name = "tet-ana/decoder-only-transformer"

# Repository erstellen, wenn noch nicht vorhanden
create_repo(repo_name, exist_ok=True)

# Model hochladen
upload_folder(
    repo_id=repo_name,
    folder_path=model_dir,
    path_in_repo=".",
    commit_message="Initial model upload"
)


genai_model.bin:   0%|          | 0.00/57.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tet-ana/decoder-only-transformer/commit/c94728e1aa7009e06b0319caed7be2d1abc4cc9d', commit_message='Initial model upload', commit_description='', oid='c94728e1aa7009e06b0319caed7be2d1abc4cc9d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tet-ana/decoder-only-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='tet-ana/decoder-only-transformer'), pr_revision=None, pr_num=None)

In [16]:



model_card = """
# Decoder-Only Transformer (Eigenbau)

Dies ist ein einfacher autoregressiver Decoder-Only Transformer, der von Grund auf in PyTorch implementiert wurde.

## Modellbeschreibung

- Transformer-Decoder-Architektur
- Trainiert auf WikiText-2-Dataset
- Tokenizer: GPT-2

## Verwendungszweck

Dieses Modell dient zur Demonstration, wie man ein autoregressives Sprachmodell selbst implementiert.

## Tags

- pytorch
- transformer
- language-modeling
- decoder-only
- education
"""

with open(os.path.join(model_dir, "README.md"), "w") as f:
    f.write(model_card)

# Model Card hochladen
upload_folder(
    repo_id=repo_name,
    folder_path=model_dir,
    path_in_repo=".",
    commit_message="Add model card"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/hf_api.py:9561: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


CommitInfo(commit_url='https://huggingface.co/tet-ana/decoder-only-transformer/commit/3a7eb9f8dae1d579340e0b4c582187740deef7df', commit_message='Add model card', commit_description='', oid='3a7eb9f8dae1d579340e0b4c582187740deef7df', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tet-ana/decoder-only-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='tet-ana/decoder-only-transformer'), pr_revision=None, pr_num=None)

In [19]:
# 🧩 Modell in den Evaluierungsmodus versetzen
model.eval()

# 🔥 Eingabetext vorbereiten (Prompt)
prompt = "When i turned back, my son"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 🪄 Anzahl der zu generierenden Token
max_new_tokens = 50
# Temperatur für Sampling (je höher, desto kreativer)
temperature = 0.8
# 🛠️ Liste, um die generierten Token zu speichern
generated_ids = inputs["input_ids"].tolist()[0]

# 🚀 Generierungsschleife
for _ in range(max_new_tokens):
    # Nur die letzten Tokens als Eingabe verwenden
    input_ids = torch.tensor([generated_ids]).to(device)

    # Vorhersage der nächsten Token-Logits
    with torch.no_grad():
        outputs = model(input_ids)

    # Logits für das letzte Token nehmen
    next_token_logits = outputs[:, -1, :]

    # Das wahrscheinlichste nächste Token auswählen
    next_token_id = torch.argmax(next_token_logits, dim=-1).item()

    # Token zur Liste hinzufügen
    generated_ids.append(next_token_id)

    # Stoppen, wenn EOS-Token generiert wurde
    if next_token_id == tokenizer.eos_token_id:
        break

# ✨ Token wieder in lesbaren Text umwandeln
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("Generated text:")
print(generated_text)


Generated text:
When i turned back, my son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son


In [20]:
import torch
import torch.nn.functional as F

# 🧩 Modell in den Evaluierungsmodus versetzen
model.eval()

# 🔥 Eingabetext vorbereiten (Prompt)
prompt = "When i turned back, my son"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 🪄 Anzahl der zu generierenden Token
max_new_tokens = 50

# Temperatur für Sampling (je höher, desto kreativer)
temperature = 0.8

# Top-k für Sampling (beschränkt Auswahl auf die Top k wahrscheinlichsten Token)
top_k = 50

# 🛠️ Liste, um die generierten Token zu speichern
generated_ids = inputs["input_ids"].tolist()[0]

# 🚀 Generierungsschleife
for _ in range(max_new_tokens):
    # Nur die letzten Tokens als Eingabe verwenden
    input_ids = torch.tensor([generated_ids]).to(device)

    # Vorhersage der nächsten Token-Logits
    with torch.no_grad():
        outputs = model(input_ids)

    # Logits für das letzte Token nehmen
    next_token_logits = outputs[:, -1, :]

    # Temperatur-Skalierung anwenden
    next_token_logits = next_token_logits / temperature

    # Top-k Sampling anwenden
    top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)

    # Wahrscheinlichkeiten berechnen
    probs = F.softmax(top_k_logits, dim=-1)

    # Zufälliges Token basierend auf den Wahrscheinlichkeiten auswählen
    next_token_id = torch.multinomial(probs, num_samples=1).item()

    # Das gewählte Token zum generierten Text hinzufügen
    generated_ids.append(top_k_indices[0, next_token_id].item())

    # Stoppen, wenn EOS-Token generiert wurde
    if generated_ids[-1] == tokenizer.eos_token_id:
        break

# ✨ Token wieder in lesbaren Text umwandeln
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("Generated text:")
print(generated_text)


Generated text:
When i turned back, my son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son son


In [19]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3


In [26]:
from transformers import PreTrainedModel, AutoTokenizer,AutoConfig, PreTrainedModel, PretrainedConfig
import torch
import torch.nn as nn
import os
from google.colab import drive
drive.mount('/content/drive')


class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=2, num_layers=2):
        super().__init__()
        # Token-Embedding
        self.embedding = nn.Embedding(vocab_size, d_model)

        # Transformer Decoder Layer
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # Lineare Projektion auf die Vokabulargröße
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids):
        # Eingabe-Token zu Embeddings
        embedded = self.embedding(input_ids).transpose(0, 1)  # (Seq_len, Batch, D_model)

        # Causal Masking erstellen
        seq_len = embedded.size(0)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(input_ids.device)

        # Transformer Decoder anwenden
        decoded = self.transformer_decoder(embedded, embedded, tgt_mask=tgt_mask)

        # Projektion auf Vokabular
        logits = self.output_layer(decoded).transpose(0, 1)  # (Batch, Seq_len, Vocab_size)
        return logits


# путь к весам (скачанных с Hugging Face)
model_path = "/content/drive/My Drive/Colab Notebooks/decoder-only-transformer/genai_model.bin"
from huggingface_hub import snapshot_download

repo_id = "tet-ana/decoder-only-transformer"  # укажи своё название репозитория
local_dir = "/content/drive/My Drive/Colab Notebooks/decoder-only-transformer"

snapshot_download(repo_id=repo_id, local_dir=local_dir)
# Загружаем конфиг
import json
with open(f"{local_dir}/config.json", "r") as f:
    config = json.load(f)

# Параметры модели из конфига
vocab_size = config["vocab_size"]
d_model = config.get("d_model", 128)
nhead = config.get("nhead", 2)
num_layers = config.get("num_layers", 2)

# Создаем модель
model = DecoderOnlyTransformer(vocab_size, d_model=d_model, nhead=nhead, num_layers=num_layers)
# загружаем веса
state_dict = torch.load(model_path, map_location="cpu")
model.load_state_dict(state_dict)

model.eval()
tokenizer = AutoTokenizer.from_pretrained(local_dir)

print("Modell erfolgreich geladen!")
# Define the device here
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Defining the device globally
import torch.nn.functional as F # Import necessary module
def generate_text(prompt, max_length=50, temperature=0.7, top_k=50):
    # Eingabetext tokenisieren und ins richtige Gerät verschieben
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    generated_ids = input_ids.tolist()[0] # Get the initial token IDs

    for _ in range(max_length):
        # Nur die letzten Tokens als Eingabe verwenden
        input_ids = torch.tensor([generated_ids]).to(device)

        # Vorhersage der nächsten Token-Logits
        with torch.no_grad():
            outputs = model(input_ids)

        # Logits für das letzte Token nehmen
        next_token_logits = outputs[:, -1, :]

        # Temperatur-Skalierung anwenden
        next_token_logits = next_token_logits / temperature

        # Top-k Sampling anwenden
        top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)

        # Wahrscheinlichkeiten berechnen
        probs = F.softmax(top_k_logits, dim=-1)

        # Zufälliges Token basierend auf den Wahrscheinlichkeiten auswählen
        next_token_id = torch.multinomial(probs, num_samples=1).item()

        # Das gewählte Token zum generierten Text hinzufügen
        generated_ids.append(top_k_indices[0, next_token_id].item())

        # Stoppen, wenn EOS-Token generiert wurde
        if generated_ids[-1] == tokenizer.eos_token_id:
            break

    # Generierten Text dekodieren
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return generated_text

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Modell erfolgreich geladen!


In [27]:
prompt = "Hallo, wie geht es dir"
generated_text = generate_text(prompt, max_length=50, temperature=0.7, top_k=50)

print(generated_text)

Hallo, wie geht es dir Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs Bulldogs
